In [8]:
 import torch
 import torch.nn as nn
 import torch.optim as optim
 import torchvision.datasets as dsets
 import torchvision.transforms as transforms
 import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)
if device == 'cuda':
  torch.cuda.manual_seed_all(1)

In [9]:
learning_rate = 0.01
training_epochs = 30
batch_size = 100

In [10]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           drop_last=True)

In [11]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.fc = nn.Linear(7*7*64,10,bias=True)
    torch.nn.init.xavier_normal(self.fc.weight)
    
  def forward(self, X):
    out = self.layer2(self.layer1(X))
    out = out.view(out.size(0), -1) # flatten
    out = self.fc(out)
    return out

model = CNN().to(device)
model

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  from ipykernel import kernelapp as app


CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [12]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
total_batch = len(data_loader)
model.train()
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    cost = criterion(model(X), Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  print("Epoch : {}, Cost : {}".format(epoch + 1, cost.item()))

Epoch : 1, Cost : 0.021993055939674377
Epoch : 2, Cost : 0.10906434804201126
Epoch : 3, Cost : 0.16623371839523315
Epoch : 4, Cost : 0.06936493515968323
Epoch : 5, Cost : 0.08605629950761795
Epoch : 6, Cost : 0.06080390885472298
Epoch : 7, Cost : 0.001118457061238587
Epoch : 8, Cost : 0.07860200107097626
Epoch : 9, Cost : 0.03408510982990265
Epoch : 10, Cost : 0.06724964827299118
Epoch : 11, Cost : 0.05550433322787285
Epoch : 12, Cost : 0.044143568724393845
Epoch : 13, Cost : 0.0034704357385635376
Epoch : 14, Cost : 0.02649528905749321
Epoch : 15, Cost : 0.04878085106611252
Epoch : 16, Cost : 0.06858653575181961
Epoch : 17, Cost : 0.06245483085513115
Epoch : 18, Cost : 0.09892137348651886
Epoch : 19, Cost : 0.018316371366381645
Epoch : 20, Cost : 0.08595362305641174
Epoch : 21, Cost : 0.001708383671939373
Epoch : 22, Cost : 0.00039490938070230186
Epoch : 23, Cost : 0.020072368904948235
Epoch : 24, Cost : 0.023334041237831116
Epoch : 25, Cost : 0.13385628163814545
Epoch : 26, Cost : 0.0

In [15]:
# test
model.eval()
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  accuracy = (torch.argmax(prediction,1) == Y_test).float().mean()
  print("acc : {}".format(accuracy))

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


acc : 0.8392999768257141


10 - 2

In [16]:
 import torch
 import torch.nn as nn
 import torch.optim as optim
 import torchvision.datasets as dsets
 import torchvision.transforms as transforms
 import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)
if device == 'cuda':
  torch.cuda.manual_seed_all(1)

# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [21]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)
        nn.ReLU()
        nn.MaxPool2d(kernel_size=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
        nn.ReLU()
        nn.MaxPool2d(kernel_size=2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1)
        nn.ReLU()
        nn.MaxPool2d(kernel_size=2,padding=1)
    )
    # 128 * 4 * 4 
    self.fc1 = nn.Linear(4*4*128,625,bias=True)
    nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = nn.Sequential(
        self.fc1,
        nn.ReLU()
        nn.Dropout(p = 0.5)
    )
    self.fc2 = nn.Linear(625,10,bias=True)
    nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer3(self.layer2(self.layer1(x)))
    out = self.fc2(self.layer4(out.view(out.size(0),-1)))
    return out

SyntaxError: ignored

In [ ]:
model = CNN().to(device)

In [19]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_batch = len(data_loader)
model.train()
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost = cost / total_batch
  print("Epoch : {}, Cost : {}".format(epoch + 1, avg_cost))

Epoch : 1, Cost : 5.0063612434314564e-05
Epoch : 2, Cost : 7.825255306670442e-05
Epoch : 3, Cost : 0.00022808001085650176
Epoch : 4, Cost : 8.084737055469304e-05
Epoch : 5, Cost : 0.00010960344661725685
Epoch : 6, Cost : 2.9082742912578396e-05
Epoch : 7, Cost : 9.671910447650589e-06
Epoch : 8, Cost : 0.00015555396385025233
Epoch : 9, Cost : 8.919455285649747e-05
Epoch : 10, Cost : 8.891185279935598e-05
Epoch : 11, Cost : 6.738373485859483e-05
Epoch : 12, Cost : 8.272546983789653e-05
Epoch : 13, Cost : 7.672470928810071e-06
Epoch : 14, Cost : 4.775029083248228e-05
Epoch : 15, Cost : 4.316477497923188e-05
Epoch : 16, Cost : 0.00010075666068587452
Epoch : 17, Cost : 4.6731947804801166e-05
Epoch : 18, Cost : 0.00012843649892602116
Epoch : 19, Cost : 1.1253879165451508e-05
Epoch : 20, Cost : 8.20615459815599e-05
Epoch : 21, Cost : 8.004592382349074e-05
Epoch : 22, Cost : 1.8822306628862862e-06
Epoch : 23, Cost : 5.5449214414693415e-05
Epoch : 24, Cost : 3.7776499084429815e-05
Epoch : 25, Co

In [20]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.8905999660491943


# Visdom

In [40]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

# pip install visdom
# python -m visdom.server

import visdom
vis = visdom.Visdom()
vis.close(env = 'main')

Setting up a new session...
Visdom python client failed to establish socket to get messages from the server. This feature is optional and can be disabled by initializing Visdom with `use_incoming_socket=False`, which will prevent waiting for this request to timeout.


In [41]:
# text
vis.text("Hello, world!",env="main") # env를 설정해줘야 나중에 한번에 종료가된다.

# image
a=torch.randn(3,200,200)
vis.image(a)

'window_3a607ecbf69214'

In [42]:
MNIST = dsets.MNIST(root="./MNIST_data", train = True,
                    transform=torchvision.transforms.ToTensor(),
                    download=True)

CIFAR10 = dsets.CIFAR10(root="./CIFAR10_data", train = True,
                        transform=torchvision.transforms.ToTensor(),
                        download=True)

Files already downloaded and verified


In [43]:
# data 하나 꺼내서 확인
data = CIFAR10.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([3, 32, 32])


'window_3a607ecce31cd6'

In [44]:
# data 하나 꺼내서 확인
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([1, 28, 28])


'window_3a607eccf01208'

In [45]:
# 여러 data 나열
data_loader = torch.utils.data.DataLoader(dataset=MNIST,
                                          batch_size=32,
                                          shuffle=True,
                                          drop_last=True)

for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [46]:
# main 종료
vis.close(env='main')

''

In [47]:
# 선 그리기
Y = torch.randn(5)
plt = vis.line(Y = Y) # 강제로 x축이 0 ~ 1이 된다.

In [48]:
X = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y = Y, X = X) # x축 설정해준다.

In [49]:
# 선 업데이트/추가

Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_3a607ecd171d06'

In [50]:
# 한 창에 여러 그래프

# line을 두개를 그리려면 Y와 똑같은 shpae의 X가 들어가야한다.
num = torch.Tensor(list(range(10))).view(-1,1)
num = torch.cat((num,num),dim=1)

# Y를 10 * 2 텐서로 넣는다.
plt = vis.line(Y=torch.randn(10,2), X=num) 

In [51]:
# Title, Legend 추가

plt = vis.line(Y=Y, X=X, opts=dict(title='Test', showlegend=True))
plt = vis.line(Y=Y, X=X, opts=dict(title='Test', legend=['1번'], showlegend=True))
plt = vis.line(Y=torch.randn(10,2), X=num, opts=dict(title='Test', legend=['1번','2번'], showlegend=True))

In [56]:
# update line function 만들기

def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num, Y=loss_value,
             win=loss_plot, update='append')

plt = vis.line(Y=torch.Tensor(1).zero_()) # pytorch의 _가 끝에 붙는 메서드는 메모리 재할당이 없다.

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [57]:
vis.close(env='main')

''

# CNN with Visdom

In [72]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

# pip install visdom
# python -m visdom.server

import visdom
vis = visdom.Visdom()
vis.close(env = 'main')

#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 32

Setting up a new session...


In [73]:
trainset = dsets.MNIST(root="./MNIST_data", train = True,
                    transform=torchvision.transforms.ToTensor(),
                    download=True)

testset = dsets.MNIST(root="./MNIST_data", train = False,
                    transform=torchvision.transforms.ToTensor(),
                    download=True)

data_loader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=32,
                                          shuffle=True,
                                          drop_last=True)

In [74]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer3(self.layer2(self.layer1(x)))
        out = out.view(out.size(0), -1)
        out = self.fc2(self.relu(self.fc1(out)))
        return out

model = CNN().to(device)

In [75]:
model = CNN().to(device)

value = torch.Tensor(1,1,28,28).to(device)
print(value.shape)
print(model(value).shape) # test code

torch.Size([1, 1, 28, 28])
torch.Size([1, 10])


In [76]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [77]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

loss_plt = vis.line(Y = torch.Tensor(1).zero_(),
                    opts=dict(title='loss_tracker', legend=['loss'],
                              showlegend=True))

In [78]:
# train
total_batch = len(data_loader)
for epoch in range(1, training_epochs + 1):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        loss = criterion(hypothesis, Y)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
    print("Epoch : {}, Loss : {}".format(epoch, loss))
    # loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

with torch.no_grad():
    X_test = testset.test_data.view(len(testset), 1, 28, 28).float().to(device)
    Y_test = testset.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())

Epoch : 1, Loss : 0.07203266769647598
Epoch : 2, Loss : 0.00018459804414305836
Epoch : 3, Loss : 0.005822018254548311
Epoch : 4, Loss : 0.011369741521775723
Epoch : 5, Loss : 0.016293026506900787
Epoch : 6, Loss : 0.08581241965293884
Epoch : 7, Loss : 0.0847674161195755
Epoch : 8, Loss : 0.0001390605902997777
Epoch : 9, Loss : 3.4272602533746976e-07
